**We have two tables-one (with 'Name', 'Day', and 'Number of Clicks columns') in SQLite database and the other (with 'User Name',	'Start Date', and	'Subscription Revenue in Past 3 Months' columns) in CRM database (the tables are presented as CRM_file and SQLite_file). At the end of each month, the sales department needs to match the usage data from website and with the CRM database for each user for further analysis. The user names in CRM and Website database are inconsistent.**

a)	Propose a Python solution that will solve the matching problem above – reads the data from source databases > matches both inputs by client name > save the result back to the database.

b)	Client names in both sources may not be consistent so exact matching/comparison may not work for 100% cases. The solution should include multiple matching strategies that can be applied to maximize the match rate.

c)	Implement the solution using Python. The implementation should address key areas.

d)	Describe how you would validate the result of the matching result during initial implementation and also during possible future enhancements of the matching algorithm by another developer.**


In this implementation file, I use fuzzywuzzy library to analyze matching process in two csv table, representing extracted files from databases.

In [1]:
## Installation of fuzzywuzzy and python-Levenshtein 
#!pip install fuzzywuzzy
#!pip install python-Levenshtein

In [2]:
# Importing the required libraries

from fuzzywuzzy import fuzz 
import pandas as pd

In [15]:
# Reading two csv files representing the source files extracted from the databases:
# SQLITE_file contains three columns with 'Name', 'Day', and 'Clicks', and CRM_file has three columns-
# 'UserName, 'StartDate', and 'Revenue'. Both the files have 45 rows of data.
# I read the tables as dataframe:
SQLITE_TABLE = pd.read_csv('SQLITE_file.txt')
CRM_TABLE = pd.read_csv('CRM_file.txt')

In [16]:
# I show the name columns of the two table for your consideration:
pd.concat([SQLITE_TABLE.Name.head(10), CRM_TABLE.UserName.head(10)], axis = 1)

,Name,UserName
0,Benjamin Harris,Ms. Samantha Wright MD
1,Sophia Lee,S. E. Adams
2,Matthew Taylor,Mrs. B. Harris PhD
3,Jacob Wilson,Oli Garcia
4,Olivia Murphy,"Prof., Biden, D"
5,Alexander Anderson,Dr. Andrew Allen
6,Saeedeh Taban,G. Rivera MD
7,Lauren Martinez,D. Murphy
8,Emily Moore,Lauren Miller
9,Ethan Simpson,Dr. A. Thompson


In [5]:
# At the very first glance the data, one can see that the following 'stopwords' list contains the
# different prefixes and suffixes, existing in the CRM_TABLE:
stopwords = ['Ms.', 'Mrs.', 'Mr.', 'PhD', 'Dr.', 'Prof.', 'MSc', 'BSc', 'MD', ',']

# Then I create a new column, named 'corrected', where the undesirable words are ommitted, and add
# it to the CRM_TABLE:
CRM_TABLE['corrected'] = CRM_TABLE.apply(lambda row: 
                                        ' '.join([word for word in row.iloc[0].split()
                                                  if word not in stopwords]), axis=1)

In [6]:
CRM_TABLE.head(10)

,UserName,StartDate,Revenue,corrected
0,Ms. Samantha Wright MD,20-10-21,670,Samantha Wright
1,S. E. Adams,20-02-26,942,S. E. Adams
2,Mrs. B. Harris PhD,22-07-14,542,B. Harris
3,Oli Garcia,18-06-09,623,Oli Garcia
4,"Prof., Biden, D",19-12-07,238,"Prof., Biden, D"
5,Dr. Andrew Allen,20-05-01,745,Andrew Allen
6,G. Rivera MD,22-07-17,227,G. Rivera
7,D. Murphy,22-05-20,157,D. Murphy
8,Lauren Miller,18-07-11,672,Lauren Miller
9,Dr. A. Thompson,22-05-02,673,A. Thompson


In [7]:
# In this implementation file, I suppose that the people (number of rows) in the tables are the same,
# and only, the name representations are different.
# To probe the matching states of the rows in two tables, I take advantage of the merge() method to
# cross join them. In this condition, evry row of SQLITE_TABLE is compared with every row of the 
# CRM_TABLE. To examine the similarity states of each pair of 'Name' (from SQLITE_TABLE) and 'corrected' 
# (from CRM_TABLE), I use Fuzzy string matching from fuzzywuzzy library, considering different states of
# matching between two given string sets based on specific algorithms.

# For cross joining the tables, I add the same zero column, 'key', to the tables.
SQLITE_TABLE['key'] = 0
CRM_TABLE['key'] = 0


# Building the cross joint table and dropping the 'key' column:
joint_table = pd.merge(SQLITE_TABLE, CRM_TABLE, on='key').drop('key', axis=1)


# Rearrangement of the columns of the joint_table for more convinience
joint_table = joint_table[['Name', 'corrected', 'UserName', 'Day', 'Clicks',
                           'StartDate', 'Revenue']]
joint_table.head(15)

,Name,corrected,UserName,Day,Clicks,StartDate,Revenue
0,Benjamin Harris,Samantha Wright,Ms. Samantha Wright MD,19-09-25,51,20-10-21,670
1,Benjamin Harris,S. E. Adams,S. E. Adams,19-09-25,51,20-02-26,942
2,Benjamin Harris,B. Harris,Mrs. B. Harris PhD,19-09-25,51,22-07-14,542
3,Benjamin Harris,Oli Garcia,Oli Garcia,19-09-25,51,18-06-09,623
4,Benjamin Harris,"Prof., Biden, D","Prof., Biden, D",19-09-25,51,19-12-07,238
5,Benjamin Harris,Andrew Allen,Dr. Andrew Allen,19-09-25,51,20-05-01,745
6,Benjamin Harris,G. Rivera,G. Rivera MD,19-09-25,51,22-07-17,227
7,Benjamin Harris,D. Murphy,D. Murphy,19-09-25,51,22-05-20,157
8,Benjamin Harris,Lauren Miller,Lauren Miller,19-09-25,51,18-07-11,672
9,Benjamin Harris,A. Thompson,Dr. A. Thompson,19-09-25,51,22-05-02,673


In [8]:
joint_table['S_R'] = joint_table.apply(lambda row: fuzz.ratio(row.iloc[0], row.iloc[1]), axis=1)
joint_table['P_R'] = joint_table.apply(lambda row: fuzz.partial_ratio(row.iloc[0], row.iloc[1]), axis=1)
joint_table['SORT_R'] = joint_table.apply(lambda row: fuzz.token_sort_ratio(row.iloc[0], row.iloc[1]), axis=1)
joint_table['SET_R'] = joint_table.apply(lambda row: fuzz.token_set_ratio(row.iloc[0], row.iloc[1]), axis=1)

# I also calculate the sum of all fuzzy malgorithms and plug the result in a new column, 'SUM', and append
# it to the joint_table
joint_table['SUM'] = joint_table[['S_R', 'P_R', 'SORT_R', 'SET_R']].sum(axis=1)
joint_table.head()

,Name,corrected,UserName,Day,Clicks,StartDate,Revenue,S_R,P_R,SORT_R,SET_R,SUM
0,Benjamin Harris,Samantha Wright,Ms. Samantha Wright MD,19-09-25,51,20-10-21,670,40,40,47,47,174
1,Benjamin Harris,S. E. Adams,S. E. Adams,19-09-25,51,20-02-26,942,23,27,33,33,116
2,Benjamin Harris,B. Harris,Mrs. B. Harris PhD,19-09-25,51,22-07-14,542,67,78,70,86,301
3,Benjamin Harris,Oli Garcia,Oli Garcia,19-09-25,51,18-06-09,623,40,50,32,32,154
4,Benjamin Harris,"Prof., Biden, D","Prof., Biden, D",19-09-25,51,19-12-07,238,27,27,37,37,128


In [9]:
# The cross joint table above considers the comparison between all the names
# included in the 'Name' and 'corrected' columns. I group the joint_table by 'Name'
# and find the index of maximum value of SUM within each group;
# The returned table is called 'table_of_max_index':
max_index = joint_table.groupby('Name')['SUM'].idxmax().reset_index()
max_index.columns = ['Name', 'Index_of_maximum_sum']
max_index

,Name,Index_of_maximum_sum
0,Alexander Anderson,248
1,Andrew Allen,995
2,Arman Roustaee,1691
3,Ashley Nelson,1319
4,Ava Martinez,1522
5,Ava Perez,1978
6,Ava Thompson,684
7,Benjamin Harris,2
8,Chloe Scott,556
9,Chloe Thompson,1558


In [10]:
# Using the max_index, I find the rows corresponding to the maximum value of SUM for each 'Name',
# indicating the best match. Afterwards, I remove the 'corrected' column, and the result is returned
# as the 'best_match_table'
best_match_table = joint_table.iloc[max_index.Index_of_maximum_sum, :].reset_index().drop(['index', 'corrected'], axis = 1)
best_match_table

,Name,UserName,Day,Clicks,StartDate,Revenue,S_R,P_R,SORT_R,SET_R,SUM
0,Alexander Anderson,A. Anderson,22-07-17,94,21-06-21,238,69,82,71,89,311
1,Andrew Allen,Dr. Andrew Allen,19-03-01,91,20-05-01,745,100,100,100,100,400
2,Arman Roustaee,Prof. A Roustaee,22-06-24,54,19-03-31,308,83,90,83,89,345
3,Ashley Nelson,Ms. Ashley Nelson,19-08-22,26,21-12-05,816,100,100,100,100,400
4,Ava Martinez,"Dr. Martinez, A",18-03-18,35,19-06-25,509,70,84,91,91,336
5,Ava Perez,"Ava, Perez, PhD",20-10-05,42,21-05-19,752,90,89,100,100,379
6,Ava Thompson,Dr. A. Thompson,22-03-03,19,22-05-02,673,87,82,91,91,351
7,Benjamin Harris,Mrs. B. Harris PhD,19-09-25,51,22-07-14,542,67,78,70,86,301
8,Chloe Scott,Dr. C. Scott,20-11-10,57,21-07-30,782,74,75,78,83,310
9,Chloe Thompson,Cloe. Thompson,20-07-07,96,21-02-10,753,93,93,96,96,378


In [11]:
# table_of_ratio_index
table_of_ratio_index = joint_table.groupby('Name').agg({'S_R': 'idxmax', 'P_R': 'idxmax',
                                             'SORT_R': 'idxmax', 'SET_R': 'idxmax'})

table_of_ratio_index.head(10)

,S_R,P_R,SORT_R,SET_R
Name,,,,
Alexander Anderson,248,248,248,248
Andrew Allen,995,995,995,995
Arman Roustaee,1691,1691,1691,1691
Ashley Nelson,1319,1319,1319,1319
Ava Martinez,1514,1522,1522,1522
Ava Perez,1978,1978,1978,1978
Ava Thompson,684,684,684,684
Benjamin Harris,2,10,2,2
Chloe Scott,556,556,556,556


In [12]:
# To see the added 'reliability' column to the table_of_ratio_index and its number of unique
# value for each name in the 'Name' column, we have:
table_of_ratio_index['reliability'] = table_of_ratio_index.nunique(axis=1)

table_of_ratio_index

,S_R,P_R,SORT_R,SET_R,reliability
Name,,,,,
Alexander Anderson,248,248,248,248,1
Andrew Allen,995,995,995,995,1
Arman Roustaee,1691,1691,1691,1691,1
Ashley Nelson,1319,1319,1319,1319,1
Ava Martinez,1514,1522,1522,1522,2
Ava Perez,1978,1978,1978,1978,1
Ava Thompson,684,684,684,684,1
Benjamin Harris,2,10,2,2,2
Chloe Scott,556,556,556,556,1


In [13]:
Suspected_entries = table_of_ratio_index[table_of_ratio_index.reliability != 1]
Suspected_entries

,S_R,P_R,SORT_R,SET_R,reliability
Name,,,,,
Ava Martinez,1514,1522,1522,1522,2
Benjamin Harris,2,10,2,2,2
David Biden,798,798,769,798,2
Jacob Martinez,1874,1882,1874,1874,2
Natalie Taylor,1092,1107,1107,1107,2


In [14]:
DOUBLED_CHECKED = best_match_table.set_index('Name').loc[Suspected_entries.index, :].reset_index()
DOUBLED_CHECKED

,Name,UserName,Day,Clicks,StartDate,Revenue,S_R,P_R,SORT_R,SET_R,SUM
0,Ava Martinez,"Dr. Martinez, A",18-03-18,35,19-06-25,509,70,84,91,91,336
1,Benjamin Harris,Mrs. B. Harris PhD,19-09-25,51,22-07-14,542,67,78,70,86,301
2,David Biden,David Wilson,21-11-04,74,21-11-22,125,70,64,43,70,247
3,Jacob Martinez,J. Martinez,19-02-25,23,22-03-23,485,80,82,83,89,334
4,Natalie Taylor,N. Taylor,20-03-23,93,20-08-18,521,70,78,73,86,307
